<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Packages" data-toc-modified-id="Packages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Packages</a></span></li><li><span><a href="#Global-parameters" data-toc-modified-id="Global-parameters-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Global parameters</a></span></li><li><span><a href="#Family-from-Genus" data-toc-modified-id="Family-from-Genus-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Family from Genus</a></span></li><li><span><a href="#With-a-NB-model" data-toc-modified-id="With-a-NB-model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>With a NB model</a></span><ul class="toc-item"><li><span><a href="#Treat-results" data-toc-modified-id="Treat-results-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Treat results</a></span></li></ul></li><li><span><a href="#With-a-CNN-model" data-toc-modified-id="With-a-CNN-model-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>With a CNN model</a></span><ul class="toc-item"><li><span><a href="#Parameters-to-load" data-toc-modified-id="Parameters-to-load-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Parameters to load</a></span></li><li><span><a href="#Loading-the-model" data-toc-modified-id="Loading-the-model-5.2"><span class="toc-item-num">5.2&nbsp;&nbsp;</span>Loading the model</a></span></li><li><span><a href="#Processing-Data" data-toc-modified-id="Processing-Data-5.3"><span class="toc-item-num">5.3&nbsp;&nbsp;</span>Processing Data</a></span><ul class="toc-item"><li><span><a href="#On-the-train" data-toc-modified-id="On-the-train-5.3.1"><span class="toc-item-num">5.3.1&nbsp;&nbsp;</span>On the train</a></span></li><li><span><a href="#On-the-test" data-toc-modified-id="On-the-test-5.3.2"><span class="toc-item-num">5.3.2&nbsp;&nbsp;</span>On the test</a></span></li><li><span><a href="#Comparisons" data-toc-modified-id="Comparisons-5.3.3"><span class="toc-item-num">5.3.3&nbsp;&nbsp;</span>Comparisons</a></span></li></ul></li><li><span><a href="#Comparing-one-rank-above" data-toc-modified-id="Comparing-one-rank-above-5.4"><span class="toc-item-num">5.4&nbsp;&nbsp;</span>Comparing one rank above</a></span><ul class="toc-item"><li><span><a href="#Retrieving--the-new-predictions-/-real-classes" data-toc-modified-id="Retrieving--the-new-predictions-/-real-classes-5.4.1"><span class="toc-item-num">5.4.1&nbsp;&nbsp;</span>Retrieving  the new predictions / real classes</a></span></li><li><span><a href="#Accuracy-Analyses" data-toc-modified-id="Accuracy-Analyses-5.4.2"><span class="toc-item-num">5.4.2&nbsp;&nbsp;</span>Accuracy Analyses</a></span></li></ul></li></ul></li></ul></div>

# Results analyses

## Packages

In [15]:
import numpy as np
import torch
from torch.autograd import Variable
from torch.nn import functional as F
import pandas as pd

In [16]:
from models.loading_model_data import main_loading_model_data
from models.cnn_preprocessing import main_preprocessing_cnn, get_homogenous_vector
from models.cnn_model import classifier_GD_2_ACM

## Global parameters 

In [17]:
sequence_origin = 'DairyDB'
primers_origin = 'Chaudhary'
taxonomy_level = 5
selected_primer = 'V4'

## Family from Genus

In [18]:
db = pd.read_csv('data\\dairydb_df.csv')

In [19]:
families = list(db['family'])
genuses = list(db['genus'])
orders = list(db['order'])

In [20]:
dict_genus_to_family = {}
dict_genus_to_order = {}
for index, genus in enumerate(genuses):
    family = families[index]
    if genus in dict_genus_to_family.keys() and family != dict_genus_to_family[genus]:
        raise ValueError('Two Families for a same Genus')
    elif genus in dict_genus_to_family.keys():
        pass
    else:
        dict_genus_to_family[genus] = family
    order = orders[index]
    if genus in dict_genus_to_order.keys() and order != dict_genus_to_order[genus]:
        raise ValueError('Two Orders for a same Genus')
    elif genus in dict_genus_to_order.keys():
        pass
    else:
        dict_genus_to_order[genus] = order    

In [21]:
dict_family_to_id = {}
dict_order_to_id = {}
for index, family in enumerate(np.unique(list(dict_genus_to_family.values()))):
    dict_family_to_id[family] = index
for index, order in enumerate(np.unique(list(dict_genus_to_order.values()))):
    dict_order_to_id[order] = index

In [23]:
train_loader, test_loader, dict_class_to_id, dict_id_to_class = main_preprocessing_cnn(
        sequence_origin=sequence_origin, 
        primers_origin=primers_origin,
        selected_primer=selected_primer, 
        taxonomy_level=taxonomy_level,
        max_size=300,
        k_mer=4
    )

## With a NB model

In [3]:
from models.naive_bayes_k import ETL_NB_k_mer
from sklearn.naive_bayes import GaussianNB

In [11]:
k = 3
X_train, X_test, y_train, y_test = ETL_NB_k_mer(k=k,
                                                sequence_origin=sequence_origin,
                                                primers_origin=primers_origin,
                                                taxonomy_level=taxonomy_level,
                                                selected_primer=selected_primer)
GNB = GaussianNB()
y_pred = GNB.fit(X_train, y_train).predict(X_test)

C:\Users\admin\Anaconda3\lib\site-packages\sklearn\utils\validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [53]:
chosen_class_test_results = y_pred
chosen_order_test_results = [dict_genus_to_order[pred] for pred in y_pred]
chosen_family_test_results = [dict_genus_to_family[pred] for pred in y_pred]

In [54]:
real_class_test_results = np.array(y_test.genus)
real_order_test_results = [dict_genus_to_order[c] for c in real_genus_test_results]
real_family_test_results = [dict_genus_to_family[c] for c in real_genus_test_results]

In [52]:
chosen_class_train_results = GNB.predict(X_train)
chosen_order_train_results = [dict_genus_to_order[c] for c in chosen_genus_train_results]
chosen_family_train_results = [dict_genus_to_family[c] for c in chosen_genus_train_results]

In [55]:
real_class_train_results = np.array(y_train.genus)
real_order_train_results = [dict_genus_to_order[c] for c in real_genus_train_results]
real_family_train_results = [dict_genus_to_family[c] for c in real_genus_train_results]

### Treat results 

In [62]:
absolute_train_errors = [chosen_class_train_results[i] != real_class_train_results[i] for i in range(len(real_class_train_results))]
print('Basic error probability: {:.2f}%'.format(((np.sum(absolute_train_errors) / len(real_class_train_results))) * 100))
absolute_train_errors = [chosen_family_train_results[i] != real_family_train_results[i] for i in range(len(real_family_train_results))]
print('Basic error probability at family level: {:.2f}%'.format(((np.sum(absolute_train_errors) / len(real_family_train_results))) * 100))
absolute_train_errors = [chosen_order_train_results[i] != real_order_train_results[i] for i in range(len(real_order_train_results))]
print('Basic error probability at order level: {:.2f}%'.format(((np.sum(absolute_train_errors) / len(real_order_train_results))) * 100))

Basic error probability: 8.52%
Basic error probability at family level: 3.06%
Basic error probability at order level: 1.87%


In [64]:
absolute_test_errors = [chosen_class_test_results[i] != real_class_test_results[i] for i in range(len(real_class_test_results))]
print('Basic error probability: {:.2f}%'.format(((np.sum(absolute_test_errors) / len(real_class_test_results))) * 100))
absolute_test_errors = [chosen_family_test_results[i] != real_family_test_results[i] for i in range(len(real_family_test_results))]
print('Basic error probability at family level: {:.2f}%'.format(((np.sum(absolute_test_errors) / len(real_family_test_results))) * 100))
absolute_test_errors = [chosen_order_test_results[i] != real_order_test_results[i] for i in range(len(real_order_test_results))]
print('Basic error probability at order level: {:.2f}%'.format(((np.sum(absolute_test_errors) / len(real_order_test_results))) * 100))

Basic error probability: 50.71%
Basic error probability at family level: 35.91%
Basic error probability at order level: 27.24%


## With a CNN model

### Parameters to load

In [3]:
parameter_config = {'sequence_origin':'DairyDB', 'primers_origin':'DairyDB', 'selected_primer':'V4', 'taxonomy_level':5, 'dimension':2, 'k_mer':1, 'vector_max_size':300, 'out_channel_1':254, 'out_channel_2':254, 'kernel_size_1_W':5, 'kernel_size_2_W':30, 'max_pool_stride_1_W':5, 'max_pool_stride_2_W':30, 'n_epochs':50, 'learning_rate':1e-3}
max_size = parameter_config['vector_max_size']
k_mer = parameter_config['k_mer']

analysis_path =  'model_results\\00018_analysis_V4_5_good\\'

In [4]:
X_train, X_test, y_train, y_test = main_loading_model_data(sequence_origin=parameter_config['sequence_origin'], 
                                                               primers_origin=parameter_config['primers_origin'], 
                                                               selected_primer=parameter_config['selected_primer'], 
                                                               taxonomy_level=parameter_config['taxonomy_level'])
train_loader, test_loader, dict_class_to_id, dict_id_to_class = main_preprocessing_cnn(
        sequence_origin=parameter_config['sequence_origin'], 
        primers_origin=parameter_config['primers_origin'],
        selected_primer=parameter_config['selected_primer'], 
        taxonomy_level=parameter_config['taxonomy_level'],
        max_size=max_size,
        k_mer=k_mer
    )

### Loading the model

In [5]:
n_out_features = len(dict_id_to_class)

conv_class = classifier_GD_2_ACM(n_out_features=n_out_features,
                                 parameter_config=parameter_config)

model_path = analysis_path + 'model.pt'

conv_class.load_state_dict(torch.load(model_path))
conv_class.eval()

classifier_GD_2_ACM(
  (conv): conv_module(
    (conv1): Conv2d(1, 254, kernel_size=(4, 5), stride=(1, 1))
    (bn1): BatchNorm2d(254, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (ReLU1): ReLU()
    (conv2): Conv2d(254, 254, kernel_size=(1, 30), stride=(1, 1))
    (bn2): BatchNorm2d(254, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (ReLU2): ReLU()
  )
  (fully_connected): fc_module(
    (avgpool): AdaptiveAvgPool2d(output_size=(1, 1))
    (fc1): Linear(in_features=254, out_features=1669, bias=True)
  )
)

### Processing Data

In [6]:
X_train, X_test, y_train, y_test
train_loader, test_loader
dict_class_to_id, dict_id_to_class

({'SLACKIA': 0,
  'GILVIMARINUS': 1,
  'DRACONIBACTERIUM': 2,
  'DESULFOMONILE': 3,
  'STREPTOBACILLUS': 4,
  'STREPTOCOCCUS': 5,
  'OSCILLIBACTER': 6,
  '[EUBACTERIUM]_NODATUM_GROUP': 7,
  'LEPTOTRICHIA': 8,
  'MICROBACTERIUM': 9,
  'KOCURIA': 10,
  'VERRUCOMICROBIA_WCHB1-41_GENUS': 11,
  'ERYSIPELOTRICHACEAE_GENUS': 12,
  'KLEBSIELLA': 13,
  'ODORIBACTER': 14,
  'TEPIDISPHAERA': 15,
  'LACTOBACILLUS': 16,
  'ACETITOMACULUM': 17,
  'PALUDIBACTER': 18,
  'NOVOSPHINGOBIUM': 19,
  'SACCHAROFERMENTANS': 20,
  'PSEUDOMONAS': 21,
  'STACKEBRANDTIA': 22,
  'STAPHYLOCOCCUS': 23,
  'MICROVIRGA': 24,
  'FINEGOLDIA': 25,
  'SUTTONELLA': 26,
  'BACILLUS': 27,
  'PSYCHROBACTER': 28,
  'BRACHYBACTERIUM': 29,
  'HALIANGIUM': 30,
  'WINOGRADSKYELLA': 31,
  'PAENIBACILLUS': 32,
  'THERMOMICROBIA_JG30-KF-CM45_GENUS': 33,
  'THERMOBACILLUS': 34,
  'MARINOMONAS': 35,
  'JEOTGALICOCCUS': 36,
  'KTEDONOBACTERIA_C0119_GENUS': 37,
  'WEEKSELLA': 38,
  'WEISSELLA': 39,
  'LEUCONOSTOC': 40,
  'ALISTIPES': 41,


In [7]:
X_test_col = X_test.iloc[:, 1]
y_test_col = y_test.iloc[:, 1]
X_train_col = X_train.iloc[:, 1] 
y_train_col = y_train.iloc[:, 1] 
new_X_test = np.array([get_homogenous_vector(X_test_col[i], max_size).transpose() for i in range(len(X_test))])
new_X_train = np.array([get_homogenous_vector(X_train_col[i], max_size).transpose() for i in range(len(X_train))])

In [8]:
def second_tour_proba(probs, threshold=0.2):
    validated_classes = [prob > threshold for prob in probs] 
    new_sum = np.sum([validated_classes[i] * probs[i] for i in range(len(probs))])
    if new_sum == 0:
        return probs
    adjusted_probs = [validated_classes[i] * probs[i] / new_sum for i in range(len(probs))]
    return adjusted_probs

In [9]:
def max_class_choice(probs):
    idx = np.argmax(probs)
    new_binary_probs = np.zeros(probs.shape)
    new_binary_probs[idx] = 1
    return new_binary_probs

#### On the train 

In [10]:
train_probs_results = np.zeros((len(X_train), n_out_features))
for index in range(len(X_train)):
    seq = X_train_col[index]
    X_sample = torch.from_numpy(new_X_train[index].astype(np.float32))
    y_sample = y_train_col[index]
    seq_variable = Variable(X_sample.unsqueeze(0))
    logit = conv_class(seq_variable)
    h_x = F.softmax(logit, dim=1).data.squeeze()
    train_probs_results[index] = h_x
    #probs, idx = h_x.sort(0, True)

In [11]:
chosen_class_train_results = np.zeros(train_probs_results.shape)
for index in range(len(train_probs_results)):
    chosen_class_train_results[index] = max_class_choice(train_probs_results[index])
train_real_classes = np.zeros(train_probs_results.shape)
for index in range(len(train_probs_results)):
    train_real_classes[index, dict_class_to_id[y_train_col[index]]] += 1

In [12]:
absolute_train_errors = np.abs(chosen_class_train_results-train_real_classes)
print('Basic error probability: {:.2f}%'.format(((np.sum(absolute_train_errors) / len(train_probs_results)) / 2) * 100))

Basic error probability: 5.43%


In [13]:
adjusted_train_probs_results = np.zeros(train_probs_results.shape)
for index in range(len(train_probs_results)):
    adjusted_train_probs_results[index] = second_tour_proba(train_probs_results[index])


In [14]:
adjusted_absolute_train_errors = np.abs(adjusted_train_probs_results-train_real_classes)
print('Adjusted error probability: {:.2f}%'.format(((np.sum(adjusted_absolute_train_errors) / len(train_probs_results)) / 2) * 100))

Adjusted error probability: 6.18%


#### On the test

In [15]:
test_probs_results = np.zeros((len(X_test), n_out_features))
for index in range(len(X_test)):
    seq = X_test_col[index]
    X_sample = torch.from_numpy(new_X_test[index].astype(np.float32))
    y_sample = y_test_col[index]
    seq_variable = Variable(X_sample.unsqueeze(0))
    logit = conv_class(seq_variable)
    h_x = F.softmax(logit, dim=1).data.squeeze()
    test_probs_results[index] = h_x
    #probs, idx = h_x.sort(0, True)

In [16]:
chosen_class_test_results = np.zeros(test_probs_results.shape)
for index in range(len(test_probs_results)):
    chosen_class_test_results[index] = max_class_choice(test_probs_results[index])
test_real_classes = np.zeros(test_probs_results.shape)
for index in range(len(test_probs_results)):
    test_real_classes[index, dict_class_to_id[y_test_col[index]]] += 1

In [17]:
absolute_test_errors = np.abs(chosen_class_test_results-test_real_classes)
print('Basic error probability: {:.2f}%'.format(((np.sum(absolute_test_errors) / len(test_probs_results)) / 2) * 100))

Basic error probability: 37.06%


In [18]:
adjusted_test_probs_results = np.zeros(test_probs_results.shape)
for index in range(len(test_probs_results)):
    adjusted_test_probs_results[index] = second_tour_proba(test_probs_results[index])

In [19]:
adjusted_absolute_test_errors = np.abs(adjusted_test_probs_results-test_real_classes)
print('Adjusted error probability: {:.2f}%'.format(((np.sum(adjusted_absolute_test_errors) / len(test_probs_results)) / 2) * 100))

Adjusted error probability: 37.55%


#### Comparisons

In [28]:
global_train_probs_results = np.sum(adjusted_train_probs_results, axis=0)
global_test_probs_results = np.sum(adjusted_test_probs_results, axis=0)
global_train_real_classes = np.sum(train_real_classes, axis=0)
global_test_real_classes = np.sum(test_real_classes, axis=0)

In [45]:
aggregated_results_df = pd.DataFrame()
aggregated_results_df['Genus'] = dict_id_to_class.values()
aggregated_results_df['Train Real Occurences']  = global_train_real_classes
aggregated_results_df['Train Model Adjusted Occurences'] = global_train_probs_results
aggregated_results_df['Test Real Occurences']  = global_test_real_classes
aggregated_results_df['Test Model Adjusted Occurences'] = global_test_probs_results
 

aggregated_results_df.to_csv('Aggregated_cnn_results.csv', index=False, sep=";")
# Then Tableau

In [67]:
aggregated_results_df.melt(id_vars='Genus', var_name='Results from', value_name='Occurences').to_csv('Melted_cnn_results.csv',index=False, sep=';')

### Comparing one rank above

#### Retrieving  the new predictions / real classes

The results used are those from chosen_class_train_results, chosen_class_test_results, adjusted_train_probs_results, adjusted_test_probs_results, compared to the real classes in train_real_classes and test_real_classes.

dict_id_to_class permet de passer du numéro id parmi les 16XX GENUS au GENUS utilisé dans les dict_genus_to_family etc.

In [106]:
def map_genus_to_higher_rank(model_results, dict_genus_to, dict_xxx_to_id, dict_id_to_class):
    higher_rank_results = np.zeros((model_results.shape[0], len(dict_xxx_to_id)))
    for index, estim in enumerate(model_results):
        for genus_id, genus_result in enumerate(estim):
            higher_rank_results[index][dict_xxx_to_id[dict_genus_to[dict_id_to_class[genus_id]]]] += genus_result
    return higher_rank_results

In [111]:
chosen_class_train_results_families = map_genus_to_higher_rank(chosen_class_train_results, dict_genus_to_family, dict_family_to_id, dict_id_to_class)
chosen_class_test_results_families = map_genus_to_higher_rank(chosen_class_test_results, dict_genus_to_family, dict_family_to_id, dict_id_to_class)
adjusted_train_probs_results_families = map_genus_to_higher_rank(adjusted_train_probs_results, dict_genus_to_family, dict_family_to_id, dict_id_to_class)
adjusted_test_probs_results_families = map_genus_to_higher_rank(adjusted_test_probs_results, dict_genus_to_family, dict_family_to_id, dict_id_to_class)
train_real_classes_families = map_genus_to_higher_rank(train_real_classes, dict_genus_to_family, dict_family_to_id, dict_id_to_class)
test_real_classes_families = map_genus_to_higher_rank(test_real_classes, dict_genus_to_family, dict_family_to_id, dict_id_to_class)

In [112]:
chosen_class_train_results_orders = map_genus_to_higher_rank(chosen_class_train_results, dict_genus_to_order, dict_order_to_id, dict_id_to_class)
chosen_class_test_results_orders = map_genus_to_higher_rank(chosen_class_test_results, dict_genus_to_order, dict_order_to_id, dict_id_to_class)
adjusted_train_probs_results_orders = map_genus_to_higher_rank(adjusted_train_probs_results, dict_genus_to_order, dict_order_to_id, dict_id_to_class)
adjusted_test_probs_results_orders = map_genus_to_higher_rank(adjusted_test_probs_results, dict_genus_to_order, dict_order_to_id, dict_id_to_class)
train_real_classes_orders = map_genus_to_higher_rank(train_real_classes, dict_genus_to_order, dict_order_to_id, dict_id_to_class)
test_real_classes_orders = map_genus_to_higher_rank(test_real_classes, dict_genus_to_order, dict_order_to_id, dict_id_to_class)

#### Accuracy Analyses

In [120]:
absolute_train_errors = np.abs(chosen_class_train_results-train_real_classes)
print('Basic error probability: {:.2f}%'.format(((np.sum(absolute_train_errors) / len(train_probs_results)) / 2) * 100))
absolute_train_errors = np.abs(chosen_class_train_results_families-train_real_classes_families)
print('Basic error probability at family level: {:.2f}%'.format(((np.sum(absolute_train_errors) / len(train_probs_results)) / 2) * 100))
absolute_train_errors = np.abs(chosen_class_train_results_orders-train_real_classes_orders)
print('Basic error probability at order level: {:.2f}%'.format(((np.sum(absolute_train_errors) / len(train_probs_results)) / 2) * 100))

Basic error probability: 5.43%
Basic error probability at family level: 2.18%
Basic error probability at order level: 1.43%


In [114]:
absolute_test_errors = np.abs(chosen_class_test_results-test_real_classes)
print('Basic error probability: {:.2f}%'.format(((np.sum(absolute_test_errors) / len(test_probs_results)) / 2) * 100))
absolute_test_errors = np.abs(chosen_class_test_results_families-test_real_classes_families)
print('Basic error probability at family level: {:.2f}%'.format(((np.sum(absolute_test_errors) / len(test_probs_results)) / 2) * 100))
absolute_test_errors = np.abs(chosen_class_test_results_orders-test_real_classes_orders)
print('Basic error probability at order level: {:.2f}%'.format(((np.sum(absolute_test_errors) / len(test_probs_results)) / 2) * 100))

Basic error probability: 37.06%
Basic error probability at family level: 18.40%
Basic error probability at order level: 13.01%


In [121]:
adjusted_absolute_train_errors = np.abs(adjusted_train_probs_results-train_real_classes)
print('Adjusted error probability: {:.2f}%'.format(((np.sum(adjusted_absolute_train_errors) / len(train_probs_results)) / 2) * 100))
adjusted_absolute_train_errors = np.abs(adjusted_train_probs_results_families-train_real_classes_families)
print('Adjusted error probability at family level: {:.2f}%'.format(((np.sum(adjusted_absolute_train_errors) / len(train_probs_results)) / 2) * 100))
adjusted_absolute_train_errors = np.abs(adjusted_train_probs_results_orders-train_real_classes_orders)
print('Adjusted error probability at order level: {:.2f}%'.format(((np.sum(adjusted_absolute_train_errors) / len(train_probs_results)) / 2) * 100))

Adjusted error probability: 6.18%
Adjusted error probability at family level: 2.63%
Adjusted error probability at order level: 1.76%


In [119]:
adjusted_absolute_test_errors = np.abs(adjusted_test_probs_results-test_real_classes)
print('Adjusted error probability: {:.2f}%'.format(((np.sum(adjusted_absolute_test_errors) / len(test_probs_results)) / 2) * 100))
adjusted_absolute_test_errors = np.abs(adjusted_test_probs_results_families-test_real_classes_families)
print('Adjusted error probability at family level: {:.2f}%'.format(((np.sum(adjusted_absolute_test_errors) / len(test_probs_results)) / 2) * 100))
adjusted_absolute_test_errors = np.abs(adjusted_test_probs_results_orders-test_real_classes_orders)
print('Adjusted error probability at order level: {:.2f}%'.format(((np.sum(adjusted_absolute_test_errors) / len(test_probs_results)) / 2) * 100))

Adjusted error probability: 37.55%
Adjusted error probability at family level: 18.53%
Adjusted error probability at order level: 12.99%
